In [1]:
%load_ext autoreload
%autoreload 2

[https://github.com/christos-c/bible-corpus](https://github.com/christos-c/bible-corpus)

In [2]:
import joblib
import xmltodict

import pandas as pd
import numpy as np

import trglobals

In [3]:
import xml.etree.ElementTree as ET

In [4]:
lang_fpath = f"../Texts/bible-corpus/bibles/Chinese.xml"
all_segs = []
root = ET.fromstring(open(lang_fpath, 'r', encoding='utf-8').read())

In [5]:
segs = list(root.iter('seg'))

In [6]:
segs[0].items()

[('id', 'b.GEN.1.1'), ('type', 'verse')]

In [7]:
def get_bible_lines(lang):
    lang_fpath = f"../Texts/bible-corpus/bibles/{lang}.xml"
    all_seg_data = []
    root = ET.fromstring(open(lang_fpath, 'r', encoding='utf-8').read())
    for n in root.iter('seg'):
        cur_seg_id = n.items()[0][1]
        cur_seg = n.text.strip()
        all_seg_data.append({'seg_id': cur_seg_id, 'seg': cur_seg})
    return all_seg_data

In [8]:
zh_seg_data = get_bible_lines("Chinese-tok")

In [9]:
zh_df = pd.DataFrame(zh_seg_data)

In [10]:
lang_map = {
    "de": "German",
    "en": "English",
    "ru": "Russian",
    "vi": "Vietnamese",
    'zh': "Chinese-tok",
}

In [11]:
bible_map = {short: pd.DataFrame(get_bible_lines(full)) for short, full in lang_map.items()}

In [12]:
bible_map.keys()

dict_keys(['de', 'en', 'ru', 'vi', 'zh'])

In [13]:
bible_map['vi'][:10]

,seg_id,seg
0,b.GEN.1.1,Ban đầu Ðức Chúa Trời dựng nên trời đất.
1,b.GEN.1.2,"Vả, đất là vô hình và trống không, sự mờ tối ở..."
2,b.GEN.1.3,Ðức Chúa Trời phán rằng: Phải có sự sáng; thì ...
3,b.GEN.1.4,"Ðức Chúa Trời thấy sự sáng là tốt lành, bèn ph..."
4,b.GEN.1.5,Ðức Chúa Trời đặt tên sự sáng là ngày; sự tối ...
5,b.GEN.1.6,Ðức Chúa Trời lại phán rằng: Phải có một khoản...
6,b.GEN.1.7,"Ngài làm nên khoảng không, phân rẽ nước ở dưới..."
7,b.GEN.1.8,Ðức Chúa Trời đặt tên khoảng không là trời. Vậ...
8,b.GEN.1.9,Ðức Chúa Trời lại phán rằng: Những nước ở dưới...
9,b.GEN.1.10,"Ðức Chúa Trời đặt tên chỗ khô cạn là đất, còn ..."


In [14]:
def get_bib_lens(bib_map):
    return {lang: len(bib) for lang, bib in bib_map.items()}

In [15]:
get_bib_lens(bible_map)

{'de': 31102, 'en': 31102, 'ru': 31102, 'vi': 31102, 'zh': 31101}

### Find the gap in zh

In [82]:
de_df = bible_map['de']
de_df['lang'] = 'de'
zh_df = bible_map['zh']
zh_df['lang'] = 'zh'

In [83]:
merged_df = de_df.merge(zh_df, on="seg_id", how='left')

In [84]:
merged_df

,seg_id,seg_x,lang_x,seg_y,lang_y
0,b.GEN.1.1,Am Anfang schuf Gott Himmel und Erde.,de,起初 神 創造 天地,zh
1,b.GEN.1.2,"Und die Erde war wüst und leer, und es war fin...",de,地是 空虛 混沌 ． 淵面 黑暗 ． 神 的 靈運 行在 水面上,zh
2,b.GEN.1.3,Und Gott sprach: Es werde Licht! und es ward L...,de,神說 、 要 有 光 、 就 有 了 光,zh
3,b.GEN.1.4,"Und Gott sah, daß das Licht gut war. Da schied...",de,神 看 光是 好 的 、 就 把 光 暗 分開了,zh
4,b.GEN.1.5,und nannte das Licht Tag und die Finsternis Na...,de,神稱光為晝 、 稱暗為夜 ． 有 晚上 、 有 早晨 、 這是 頭一日,zh
...,...,...,...,...,...
31097,b.REV.22.17,Und der Geist und die Braut sprechen: Komm! Un...,de,聖靈 和 新婦 都 說來 。 聽見 的 人 也 該說來 。 口渴 的 人 也 當來 ． 願意...,zh
31098,b.REV.22.18,"Ich bezeuge allen, die da hören die Worte der ...",de,我 向 一切 聽見這書 上 豫 言 的 作見證 、 若 有 人 在 這豫 言 上 加添 甚 ...,zh
31099,b.REV.22.19,Und so jemand davontut von den Worten des Buch...,de,這書 上 的 豫言 、 若 有 人 刪 去 甚麼 、 神必從這書 上 所 寫 的 生命樹 、...,zh
31100,b.REV.22.20,"Es spricht, der solches bezeugt: Ja, ich komme...",de,證明 這事 的 說 、 是 了 ． 我 必 快來 。 阿們 。 主耶 穌阿 、 我願你來,zh


In [85]:
de_only = merged_df[pd.isna(merged_df['seg_y'])]
de_only

,seg_id,seg_x,lang_x,seg_y,lang_y
1983,b.EXO.16.36,Ein Gomer aber ist der zehnte Teil eines Epha.,de,NaN,NaN
10983,b.1CH.22.19,"So richtet nun euer Herz und eure Seele, den H...",de,NaN,NaN
26381,b.JOH.7.53,Und ein jeglicher ging also heim.,de,NaN,NaN


In [86]:
rev_merged_df = zh_df.merge(de_df, on='seg_id', how='left')

In [89]:
zh_only = list(rev_merged_df[pd.isna(rev_merged_df['seg_y'])]['seg_id'].values)
zh_only

['b.1CH.21.31', 'b.3JO.1.15']

In [91]:
merged_df.rename(columns={'seg_x':'de', 'seg_y': 'zh'}, inplace=True)

In [93]:
merged_df.drop(columns=['lang_x','lang_y'], inplace=True)

In [95]:
merged_df = merged_df[["de","zh","seg_id"]]

In [96]:
merged_df

,de,zh,seg_id
0,Am Anfang schuf Gott Himmel und Erde.,起初 神 創造 天地,b.GEN.1.1
1,"Und die Erde war wüst und leer, und es war fin...",地是 空虛 混沌 ． 淵面 黑暗 ． 神 的 靈運 行在 水面上,b.GEN.1.2
2,Und Gott sprach: Es werde Licht! und es ward L...,神說 、 要 有 光 、 就 有 了 光,b.GEN.1.3
3,"Und Gott sah, daß das Licht gut war. Da schied...",神 看 光是 好 的 、 就 把 光 暗 分開了,b.GEN.1.4
4,und nannte das Licht Tag und die Finsternis Na...,神稱光為晝 、 稱暗為夜 ． 有 晚上 、 有 早晨 、 這是 頭一日,b.GEN.1.5
...,...,...,...
31097,Und der Geist und die Braut sprechen: Komm! Un...,聖靈 和 新婦 都 說來 。 聽見 的 人 也 該說來 。 口渴 的 人 也 當來 ． 願意...,b.REV.22.17
31098,"Ich bezeuge allen, die da hören die Worte der ...",我 向 一切 聽見這書 上 豫 言 的 作見證 、 若 有 人 在 這豫 言 上 加添 甚 ...,b.REV.22.18
31099,Und so jemand davontut von den Worten des Buch...,這書 上 的 豫言 、 若 有 人 刪 去 甚麼 、 神必從這書 上 所 寫 的 生命樹 、...,b.REV.22.19
31100,"Es spricht, der solches bezeugt: Ja, ich komme...",證明 這事 的 說 、 是 了 ． 我 必 快來 。 阿們 。 主耶 穌阿 、 我願你來,b.REV.22.20


### And compute Chinese TMD

In [99]:
trglobals.init_embedding_model()

xlang_model initialized


In [100]:
merged_df['tmd'] = trglobals.compute_tmd_df(merged_df)

100%|██████████| 31102/31102 [04:40<00:00, 110.90it/s]
C:\Users\jpjac\AppData\Local\Temp\ipykernel_3404\3034185770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['tmd'] = trglobals.compute_tmd_df(merged_df)


In [101]:
merged_df

,de,zh,seg_id,tmd
0,Am Anfang schuf Gott Himmel und Erde.,起初 神 創造 天地,b.GEN.1.1,1.014973
1,"Und die Erde war wüst und leer, und es war fin...",地是 空虛 混沌 ． 淵面 黑暗 ． 神 的 靈運 行在 水面上,b.GEN.1.2,1.087618
2,Und Gott sprach: Es werde Licht! und es ward L...,神說 、 要 有 光 、 就 有 了 光,b.GEN.1.3,1.041614
3,"Und Gott sah, daß das Licht gut war. Da schied...",神 看 光是 好 的 、 就 把 光 暗 分開了,b.GEN.1.4,1.011653
4,und nannte das Licht Tag und die Finsternis Na...,神稱光為晝 、 稱暗為夜 ． 有 晚上 、 有 早晨 、 這是 頭一日,b.GEN.1.5,1.025686
...,...,...,...,...
31097,Und der Geist und die Braut sprechen: Komm! Un...,聖靈 和 新婦 都 說來 。 聽見 的 人 也 該說來 。 口渴 的 人 也 當來 ． 願意...,b.REV.22.17,1.067367
31098,"Ich bezeuge allen, die da hören die Worte der ...",我 向 一切 聽見這書 上 豫 言 的 作見證 、 若 有 人 在 這豫 言 上 加添 甚 ...,b.REV.22.18,1.067616
31099,Und so jemand davontut von den Worten des Buch...,這書 上 的 豫言 、 若 有 人 刪 去 甚麼 、 神必從這書 上 所 寫 的 生命樹 、...,b.REV.22.19,1.115010
31100,"Es spricht, der solches bezeugt: Ja, ich komme...",證明 這事 的 說 、 是 了 ． 我 必 快來 。 阿們 。 主耶 穌阿 、 我願你來,b.REV.22.20,0.960952


In [106]:
zh_mean = merged_df[~np.isinf(merged_df['tmd'])]['tmd'].mean()
zh_mean

1.0696386263149718

In [107]:
joblib.dump(zh_mean, './lang_stats/bible_zh_mean.pkl')

['./lang_stats/bible_zh_mean.pkl']

### Vietnamese

In [21]:
de_df = bible_map['de']
de_df['lang'] = 'de'
vi_df = bible_map['vi']
vi_df['lang'] = 'vi'

In [22]:
merged_df = de_df.merge(vi_df, on="seg_id", how='left')

In [23]:
merged_df

,seg_id,seg_x,lang_x,seg_y,lang_y
0,b.GEN.1.1,Am Anfang schuf Gott Himmel und Erde.,de,Ban đầu Ðức Chúa Trời dựng nên trời đất.,vi
1,b.GEN.1.2,"Und die Erde war wüst und leer, und es war fin...",de,"Vả, đất là vô hình và trống không, sự mờ tối ở...",vi
2,b.GEN.1.3,Und Gott sprach: Es werde Licht! und es ward L...,de,Ðức Chúa Trời phán rằng: Phải có sự sáng; thì ...,vi
3,b.GEN.1.4,"Und Gott sah, daß das Licht gut war. Da schied...",de,"Ðức Chúa Trời thấy sự sáng là tốt lành, bèn ph...",vi
4,b.GEN.1.5,und nannte das Licht Tag und die Finsternis Na...,de,Ðức Chúa Trời đặt tên sự sáng là ngày; sự tối ...,vi
...,...,...,...,...,...
31097,b.REV.22.17,Und der Geist und die Braut sprechen: Komm! Un...,de,Thánh Linh và vợ mới cùng nói: Hãy đến! Kẻ nào...,vi
31098,b.REV.22.18,"Ich bezeuge allen, die da hören die Worte der ...",de,Tôi ngỏ cho kẻ nào nghe lời tiên tri trong sác...,vi
31099,b.REV.22.19,Und so jemand davontut von den Worten des Buch...,de,Và kẻ nào bớt điều gì trong những lời ở sách t...,vi
31100,b.REV.22.20,"Es spricht, der solches bezeugt: Ja, ich komme...",de,"Ðấng làm chứng cho những điều ấy, phán rằng: P...",vi


In [24]:
merged_df.rename(columns={'seg_x':'de', 'seg_y': 'vi'}, inplace=True)

In [25]:
merged_df.drop(columns=['lang_x', 'lang_y'], inplace=True)

In [26]:
merged_df

,seg_id,de,vi
0,b.GEN.1.1,Am Anfang schuf Gott Himmel und Erde.,Ban đầu Ðức Chúa Trời dựng nên trời đất.
1,b.GEN.1.2,"Und die Erde war wüst und leer, und es war fin...","Vả, đất là vô hình và trống không, sự mờ tối ở..."
2,b.GEN.1.3,Und Gott sprach: Es werde Licht! und es ward L...,Ðức Chúa Trời phán rằng: Phải có sự sáng; thì ...
3,b.GEN.1.4,"Und Gott sah, daß das Licht gut war. Da schied...","Ðức Chúa Trời thấy sự sáng là tốt lành, bèn ph..."
4,b.GEN.1.5,und nannte das Licht Tag und die Finsternis Na...,Ðức Chúa Trời đặt tên sự sáng là ngày; sự tối ...
...,...,...,...
31097,b.REV.22.17,Und der Geist und die Braut sprechen: Komm! Un...,Thánh Linh và vợ mới cùng nói: Hãy đến! Kẻ nào...
31098,b.REV.22.18,"Ich bezeuge allen, die da hören die Worte der ...",Tôi ngỏ cho kẻ nào nghe lời tiên tri trong sác...
31099,b.REV.22.19,Und so jemand davontut von den Worten des Buch...,Và kẻ nào bớt điều gì trong những lời ở sách t...
31100,b.REV.22.20,"Es spricht, der solches bezeugt: Ja, ich komme...","Ðấng làm chứng cho những điều ấy, phán rằng: P..."


In [29]:
# Rearrange the cols
merged_df = merged_df[["de","vi","seg_id"]].copy()

In [30]:
merged_df

,de,vi,seg_id
0,Am Anfang schuf Gott Himmel und Erde.,Ban đầu Ðức Chúa Trời dựng nên trời đất.,b.GEN.1.1
1,"Und die Erde war wüst und leer, und es war fin...","Vả, đất là vô hình và trống không, sự mờ tối ở...",b.GEN.1.2
2,Und Gott sprach: Es werde Licht! und es ward L...,Ðức Chúa Trời phán rằng: Phải có sự sáng; thì ...,b.GEN.1.3
3,"Und Gott sah, daß das Licht gut war. Da schied...","Ðức Chúa Trời thấy sự sáng là tốt lành, bèn ph...",b.GEN.1.4
4,und nannte das Licht Tag und die Finsternis Na...,Ðức Chúa Trời đặt tên sự sáng là ngày; sự tối ...,b.GEN.1.5
...,...,...,...
31097,Und der Geist und die Braut sprechen: Komm! Un...,Thánh Linh và vợ mới cùng nói: Hãy đến! Kẻ nào...,b.REV.22.17
31098,"Ich bezeuge allen, die da hören die Worte der ...",Tôi ngỏ cho kẻ nào nghe lời tiên tri trong sác...,b.REV.22.18
31099,Und so jemand davontut von den Worten des Buch...,Và kẻ nào bớt điều gì trong những lời ở sách t...,b.REV.22.19
31100,"Es spricht, der solches bezeugt: Ja, ich komme...","Ðấng làm chứng cho những điều ấy, phán rằng: P...",b.REV.22.20


In [31]:
merged_df['tmd'] = trglobals.compute_tmd_df(merged_df)

100%|██████████| 31102/31102 [02:59<00:00, 173.29it/s]


In [34]:
vi_noninf_df = merged_df[~np.isinf(merged_df['tmd'])].copy()

In [37]:
vi_mean = vi_noninf_df['tmd'].mean()
vi_mean

1.0661334938727538

In [38]:
joblib.dump(vi_mean, "./lang_stats/bible_vi_mean.pkl")

['./lang_stats/bible_vi_mean.pkl']

### And now compute the TMDs

In [32]:
import trglobals
from tqdm import tqdm
tqdm.pandas()

In [17]:
trglobals.init_embedding_model()

xlang_model initialized


In [16]:
en_df = bib_df[['de','en']].copy()

In [17]:
lang1 = 'de'
lang2 = 'en'
compute_tmd_partial = lambda cur_row: trglobals.compute_tmd(cur_row, lang1, lang2)

In [18]:
en_df['tmd'] = en_df.progress_apply(compute_tmd_partial, axis=1)

100%|██████████| 31102/31102 [02:24<00:00, 215.72it/s]


In [19]:
en_df.to_csv("../Texts_Aligned/de_bib.en_bib.csv")

### English (non-inf) mean

In [42]:
en_noninf_df = en_df[~np.isinf(en_df['tmd'])].copy()

In [45]:
bib_en_mean = en_noninf_df['tmd'].mean()
bib_en_mean

0.8841099582814969

In [46]:
joblib.dump(bib_en_mean, './lang_stats/bible_en_mean.pkl')

['./lang_stats/bible_en_mean.pkl']

### Russian mean

In [20]:
lang1 = 'de'
lang2 = 'ru'
ru_df = bib_df[[lang1,lang2]].copy()

In [21]:
compute_tmd_partial = lambda cur_row: trglobals.compute_tmd(cur_row, lang1, lang2)
ru_df['tmd'] = ru_df.progress_apply(compute_tmd_partial, axis=1)

100%|██████████| 31102/31102 [01:52<00:00, 276.25it/s]


In [22]:
ru_df

,de,ru,tmd
0,Am Anfang schuf Gott Himmel und Erde.,В начале сотворил Бог небо и землю.,0.612842
1,"Und die Erde war wüst und leer, und es war fin...","Земля же была безвидна и пуста, и тьма над без...",0.917743
2,Und Gott sprach: Es werde Licht! und es ward L...,И сказал Бог: да будет свет. И сталсвет.,0.858099
3,"Und Gott sah, daß das Licht gut war. Da schied...","И увидел Бог свет, что он хорош, и отделил Бог...",0.785763
4,und nannte das Licht Tag und die Finsternis Na...,"И назвал Бог свет днем, а тьму ночью. И был ве...",0.775576
...,...,...,...
31097,Und der Geist und die Braut sprechen: Komm! Un...,И Дух и невеста говорят: прииди! И слышавший д...,0.965954
31098,"Ich bezeuge allen, die da hören die Worte der ...",И я также свидетельствую всякому слышащему сло...,0.892177
31099,Und so jemand davontut von den Worten des Buch...,и если кто отнимет что от слов книги пророчест...,0.930664
31100,"Es spricht, der solches bezeugt: Ja, ich komme...","Свидетельствующий сие говорит: ей, гряду скоро...",0.938942


In [23]:
ru_df.mean(numeric_only=True)

tmd    inf
dtype: float64

In [24]:
ru_df.to_csv("../Texts_Aligned/de_bib.ru_bib.csv")

In [25]:
import pandas as pd
import numpy as np

In [26]:
ru_df = pd.read_csv("../Texts_Aligned/de_bib.ru_bib.csv")

In [27]:
ru_df

,Unnamed: 0,de,ru,tmd
0,0,Am Anfang schuf Gott Himmel und Erde.,В начале сотворил Бог небо и землю.,0.612842
1,1,"Und die Erde war wüst und leer, und es war fin...","Земля же была безвидна и пуста, и тьма над без...",0.917743
2,2,Und Gott sprach: Es werde Licht! und es ward L...,И сказал Бог: да будет свет. И сталсвет.,0.858099
3,3,"Und Gott sah, daß das Licht gut war. Da schied...","И увидел Бог свет, что он хорош, и отделил Бог...",0.785763
4,4,und nannte das Licht Tag und die Finsternis Na...,"И назвал Бог свет днем, а тьму ночью. И был ве...",0.775576
...,...,...,...,...
31097,31097,Und der Geist und die Braut sprechen: Komm! Un...,И Дух и невеста говорят: прииди! И слышавший д...,0.965954
31098,31098,"Ich bezeuge allen, die da hören die Worte der ...",И я также свидетельствую всякому слышащему сло...,0.892177
31099,31099,Und so jemand davontut von den Worten des Buch...,и если кто отнимет что от слов книги пророчест...,0.930664
31100,31100,"Es spricht, der solches bezeugt: Ja, ich komme...","Свидетельствующий сие говорит: ей, гряду скоро...",0.938942


In [28]:
ru_df_finite = ru_df[~np.isinf(ru_df['tmd'])].copy()

In [29]:
ru_mean = ru_df_finite['tmd'].mean()

In [30]:
ru_mean

0.929112058449112

In [33]:
joblib.dump(ru_mean, './lang_stats/bible_ru_mean.pkl')

['./lang_stats/bible_ru_mean.pkl']

### Vietnamese mean

In [19]:
lang1 = 'de'
lang2 = 'vi'
vi_df = bib_df[[lang1,lang2]].copy()

NameError: name 'bib_df' is not defined